In [18]:
import os
import zipfile
#from zipfile import ZipFile
import pandas as pd

In [8]:
os.listdir('./alertaRio')

['DadosPluviometricos_2018.zip',
 'DadosPluviometricos_2019.zip',
 'DadosPluviometricos_2020.zip',
 'DadosPluviometricos_2021.zip',
 'DadosPluviometrico_2022.zip']

In [5]:
zip_path = './alertaRio/'

In [3]:
for arquivo in os.listdir(zip_path):
    if '.zip' in arquivo:
        with zipfile.ZipFile(zip_path + arquivo, 'r') as zipObj:
            zipObj.extractall(zip_path+'extracted_files')

In [7]:
def reading_txts(path):
    dataex = pd.read_csv(path, header=2, sep='\s+')
    dataex = dataex.rename(columns={'HBV':'15min', '15':'01h', 'min':'04h', '01':'24h', 'h':'96h'}).dropna(how='all', axis=1)

    return dataex

In [2]:
import pandas as pd
dataex = pd.read_csv('./alertaRio/extracted_files/jardim_botanico_202108_Plv.txt', header=2, sep='\s+')

In [3]:
dataex.rename(columns={'HBV':'15min', '15':'01h', 'min':'04h', '01':'24h', 'h':'96h'}).dropna(how='all', axis=1)

,Dia,Hora,15min,01h,04h,24h,96h
0,01/08/2021,00:00:00,0.0,0.0,0.0,0.0,40.8
1,01/08/2021,00:15:00,0.0,0.0,0.0,0.0,40.8
2,01/08/2021,00:30:00,0.0,0.0,0.0,0.0,40.8
3,01/08/2021,00:45:00,0.0,0.0,0.0,0.0,40.8
4,01/08/2021,01:00:00,0.0,0.0,0.0,0.0,40.8
...,...,...,...,...,...,...,...
2971,31/08/2021,22:45:00,0.0,0.0,0.8,9.6,32.2
2972,31/08/2021,23:00:00,0.0,0.0,0.8,9.6,32.2
2973,31/08/2021,23:15:00,0.0,0.0,0.2,9.6,32.2
2974,31/08/2021,23:30:00,0.0,0.0,0.2,9.6,32.2


In [8]:
data_list = []
for txt in os.listdir(zip_path+'extracted_files'):
    data_list.append(reading_txts(zip_path+'extracted_files/'+txt))


In [9]:
data_list[-1]
precip18_22 = pd.concat(data_list)
precip18_22

,Dia,Hora,15min,01h,04h,24h,96h,04
0,01/01/2018,00:00:00,HBV,0.0,0.0,0.0,0.0,10.8
1,01/01/2018,00:15:00,HBV,0.0,0.0,0.0,0.0,10.8
2,01/01/2018,00:30:00,HBV,0.0,0.0,0.0,0.0,10.8
3,01/01/2018,00:45:00,HBV,0.0,0.0,0.0,0.0,10.8
4,01/01/2018,01:00:00,HBV,0.0,0.0,0.0,0.0,10.8
...,...,...,...,...,...,...,...,...
1444,16/11/2022,01:00:00,0.0,0.0,0.0,1.6,2.8,NaN
1445,16/11/2022,01:15:00,0.0,0.0,0.0,1.6,2.8,NaN
1446,16/11/2022,01:30:00,0.0,0.0,0.0,1.6,2.6,NaN
1447,16/11/2022,01:45:00,0.0,0.0,0.0,1.6,2.6,NaN


In [27]:
HBV_problem = precip18_22.where(precip18_22['15min']=='HBV').dropna(how='all')
HBV_problem.columns = ['Dia', 'Hora', 'HBV','15min', '01h', '04h', '24h', '96h']
HBV_problem = HBV_problem.drop('HBV', axis=1)

adjusting_precip  = precip18_22.mask(precip18_22['15min']=='HBV').dropna(how='all')
adjusting_precip = adjusting_precip.drop('04', axis=1)

adjusted18_22 = pd.concat([HBV_problem, adjusting_precip])

In [23]:
#14976 - hbv problem
#154092 - adjusting_precip
#169068 - precip full data
154092 + 14976

169068

In [28]:
adjusted18_22

,Dia,Hora,15min,01h,04h,24h,96h
0,01/01/2018,00:00:00,0.0,0.0,0.0,0.0,10.8
1,01/01/2018,00:15:00,0.0,0.0,0.0,0.0,10.8
2,01/01/2018,00:30:00,0.0,0.0,0.0,0.0,10.8
3,01/01/2018,00:45:00,0.0,0.0,0.0,0.0,10.8
4,01/01/2018,01:00:00,0.0,0.0,0.0,0.0,10.8
...,...,...,...,...,...,...,...
1444,16/11/2022,01:00:00,0.0,0.0,0.0,1.6,2.8
1445,16/11/2022,01:15:00,0.0,0.0,0.0,1.6,2.8
1446,16/11/2022,01:30:00,0.0,0.0,0.0,1.6,2.6
1447,16/11/2022,01:45:00,0.0,0.0,0.0,1.6,2.6


In [29]:
adjusted18_22['datetime'] = adjusted18_22['Dia'] + ' ' + adjusted18_22['Hora']
adjusted18_22['datetime'] = pd.to_datetime(adjusted18_22['datetime'])

,Dia,Hora,15min,01h,04h,24h,96h,datetime
0,01/01/2018,00:00:00,0.0,0.0,0.0,0.0,10.8,01/01/2018 00:00:00
1,01/01/2018,00:15:00,0.0,0.0,0.0,0.0,10.8,01/01/2018 00:15:00
2,01/01/2018,00:30:00,0.0,0.0,0.0,0.0,10.8,01/01/2018 00:30:00
3,01/01/2018,00:45:00,0.0,0.0,0.0,0.0,10.8,01/01/2018 00:45:00
4,01/01/2018,01:00:00,0.0,0.0,0.0,0.0,10.8,01/01/2018 01:00:00
...,...,...,...,...,...,...,...,...
1444,16/11/2022,01:00:00,0.0,0.0,0.0,1.6,2.8,16/11/2022 01:00:00
1445,16/11/2022,01:15:00,0.0,0.0,0.0,1.6,2.8,16/11/2022 01:15:00
1446,16/11/2022,01:30:00,0.0,0.0,0.0,1.6,2.6,16/11/2022 01:30:00
1447,16/11/2022,01:45:00,0.0,0.0,0.0,1.6,2.6,16/11/2022 01:45:00
